# PyQt5私房手册-多线程

## 深入理解PyQt5的多线程

PyQt5的多线程比较复杂，先看以下几篇文章，PyQt5的机制是否一样，还不确定：
- [QT之深入理解QThread](https://www.cnblogs.com/findumars/p/5185128.html)
- [QThread与多线程](https://blog.csdn.net/Amnes1a/article/details/70171519)
- [Qt线程与事件循环的正确用法](https://blog.csdn.net/youlezhe/article/details/60755535)

简单来说，有2种方法可以实现多线程。一种是`PyQt4`的时候，使用继承`QThread`的方式，一种是`PyQt5`推荐的，使用一个继承自`QObject`的子类的`moveToThread`方法。看个例子，用2种方法实现延迟3秒出现"开始"的`Label`：

1. 继承`QThread`的写法：

```python
class Example(QWidget):

    def __init__(self):
        super().__init__()
        self.setGeometry(300, 300, 300, 200)
        self.setWindowTitle('Tooltips')

        self.lbl = QLabel(self)
        self.lbl.resize(100, 50)
        self.lbl.move(50, 50)

        self.thread = MyThread()
        self.thread.signal.connect(self.set_text)
        self.thread.start()

    def set_text(self, text):
        self.lbl.setText(text)


class MyThread(QThread):
    signal = pyqtSignal(str)

    def run(self):
        sleep(3)
        self.signal.emit("开始")
```


2. `moveToThread`的写法

```python
class Example(QWidget):

    def __init__(self):
        super().__init__()
        self.setGeometry(300, 300, 300, 200)

        self.lbl = QLabel(self)
        self.lbl.resize(100, 50)
        self.lbl.move(50, 50)

        self.job = MyJob()
        self.thread = QThread()
        self.job.moveToThread(self.thread)
        self.job.start_work_signal.connect(self.set_text)
        self.job.stop_thread_signal.connect(self.stop_thread)
        self.thread.started.connect(self.job.start)
        self.thread.start()

    def set_text(self, text):
        self.lbl.setText(text)

    def stop_thread(self):
        self.thread.quit()


class MyJob(QObject):
    start_work_signal = pyqtSignal(str)
    stop_thread_signal = pyqtSignal()

    def start(self):
        sleep(3)
        self.start_work_signal.emit("开始")
        self.stop_thread_signal.emit()
```

要注意，这里不管是第一种方法的`self.thread`还是第二种方法的`self.job`，都需要设置为实例属性而不能是局部变量，即不能直接使用`thread=QThread()`或者`job=MyJob()`这样的写法。否则会抛出`QThread: Destroyed while thread is still running`错误，或者不起作用，原因都是`connect`并不会增加`thread`或者`QObject`的引用计数，函数执行完毕，局部变量就会被`Python`回收掉。

另外，使用第二种`moveToThread`方式，线程不会主动结束，相当于在第一种继承方式的`run`方法中，执行了`self.exec()`，开启了事件循环，因此，如果要线程结束，还需要像例子中那样调用`self.thread.quit()`方法。

## 常见错误

### `QThread: Destroyed while thread is still running`错误

当Python对线程进行垃圾收集时，就会出现这个问题。比如有下面的代码：
```python
def btn_click(self):
    thread = QThread()
```
就有可能会报错，因为在`btn_click`函数创建了`thread`以后会继续执行，当函数执行完以后，`thread`由于是局部变量，此时`python`会对其进行垃圾回收，而如果线程此时还没有执行完，就会抛出`QThread: Destroyed while thread is still running`错误。

解决方法很简单，将局部变量`thread`变成实例属性即可，保存对线程的引用，避免被回收：
```python
def btn_click(self):
    self.thread = QThread()
```

但还有个问题要注意：

如果同时存在多个线程，那么需要将对象存储在一个列表中。当一个线程完成时，需要从列表中清除对象(这样它们就被垃圾收集了)，这样就不会给应用程序带来内存泄漏。

### `QObject::connect: Cannot queue arguments of type 'QTextCursor'`错误

与其他一些GUI工具包(例如Java Swing)一样，`PyQt`不允许主线程以外的线程访问`QtGui`类。如以下的代码：
```python
class Worker(QThread):
    def run():
        self.textbrowser.append("text")
```
`QTextBrowser`属于`GtGui`类，不能在辅助线程里面访问。解决方法是使用信号和槽进行通信：
```python
class MyThread(QtCore.QThread):
    updated = QtCore.pyqtSignal(str)

    def run( self ):
        # do some functionality
        for i in range(10000):
            self.updated.emit(str(i))

class Windows(QtGui.QWidget):
    def __init__( self, parent = None ):
        super(Windows, self).__init__(parent)

        self._thread = MyThread(self)
        self._thread.updated.connect(self.updateText)

        # create a line edit and a button

        self._button.clicked.connect(self._thread.start)

    def updateText( self, text ):
        self.widget.setText(text)
```

附：[stackovervlow上的讨论](https://stackoverflow.com/questions/2104779/qobject-qplaintextedit-multithreading-issues?r=SearchResults)